In [1]:
import pandas as pd
import numpy as np

monthly_prices = pd.read_csv("../data/processed/prices_monthly.csv", index_col=0, parse_dates=True)
monthly_returns = pd.read_csv("../data/processed/returns_monthly.csv", index_col=0, parse_dates=True)
ff5 = pd.read_csv("../data/processed/ff5_monthly_clean.csv", index_col=0, parse_dates=True)
panel = pd.read_csv("../data/processed/panel_monthly_with_ff5.csv", index_col=[0,1], parse_dates=[0])

ff5.index = ff5.index + pd.offsets.MonthEnd(0)

# Momentum
momentum_12_1 = monthly_prices.shift(1) / monthly_prices.shift(13) - 1
momentum_6_1  = monthly_prices.shift(1) / monthly_prices.shift(7)  - 1
momentum_3_1  = monthly_prices.shift(1) / monthly_prices.shift(4)  - 1

# 3 month volatility of returns
vol_3m = monthly_returns.rolling(3).std()

X = (momentum_12_1.stack().rename("momentum_12_1")
     .to_frame().join(momentum_6_1.stack().rename("momentum_6_1"))
     .join(momentum_3_1.stack().rename("momentum_3_1"))
     .join(vol_3m.stack().rename("vol_3m")))

# Lag FF5 factors
ff5 = pd.read_csv("../data/processed/ff5_monthly_clean.csv", index_col=0, parse_dates=True)
ff5.index = ff5.index + pd.offsets.MonthEnd(0)
ff5_lag = ff5.shift(1).add_suffix("_lag1")

X = X.join(ff5_lag, how="left", on=X.index.get_level_values(0))

# Target: next month excess return
panel = pd.read_csv("../data/processed/panel_monthly_with_ff5.csv", index_col=[0,1], parse_dates=[0])
y_next  = panel["excess_ret"].shift(-1).rename("y_next")
data = pd.concat([X, y_next], axis=1).dropna()

data.to_csv("../data/processed/ml_feature_matrix.csv", index=True)
print("Feature matrix shape:", data.shape)
data.head()

Feature matrix shape: (1189, 12)


key_0  momentum_12_1  momentum_6_1  momentum_3_1  \
Date                                                                    
2015-02-28 AAPL 2015-02-28       0.672751      0.236959      0.089519   
           AMZN 2015-02-28      -0.011598      0.132720      0.160643   
           GOOG 2015-02-28      -0.093873     -0.064870     -0.043929   
           HD   2015-02-28       0.388100      0.304252      0.075868   
           JPM  2015-02-28       0.008552     -0.044589     -0.095076   

                   vol_3m  Mkt-RF_lag1  SMB_lag1  HML_lag1  RMW_lag1  \
Date                                                                   
2015-02-28 AAPL  0.090495      -0.0309   -0.0093   -0.0345    0.0158   
           AMZN  0.115630      -0.0309   -0.0093   -0.0345    0.0158   
           GOOG  0.036820      -0.0309   -0.0093   -0.0345    0.0158   
           HD    0.052728      -0.0309   -0.0093   -0.0345    0.0158   
           JPM   0.128209      -0.0309   -0.0093   -0.0345    0.0158   

                 CMA_lag1  RF_lag1    y_next  
Date                                          
2015-02-28 AAPL   -0.0164      0.0  0.072293  
           AMZN   -0.0164      0.0  0.044676  
           GOOG   -0.0164      0.0  0.098928  
           HD     -0.0164      0.0  0.126885  
           JPM    -0.0164      0.0  0.040311